# Planteamiento
El sitio web de dónde se obtienen los datos cuenta con bloqueo de cloud fire por lo que se usará cloudscraper

In [42]:
import cloudscraper

scraper = cloudscraper.create_scraper()
url = 'https://es.cointelegraph.com'
page = scraper.get(url+'/tags/bitcoin')

Para tener un mayor control con los datos que se obtienen, únicamente se tomarán en cuenta los artículos que pertenezcan a la categoría noticias

In [43]:
from bs4 import BeautifulSoup
soup_page = BeautifulSoup(page.text)
_links = soup_page.find_all('a',{'class':['post-card-inline__title-link']})
_links = [url+x['href'] for x in _links]
categorias = soup_page.find_all('span', {'class':['post-card-inline__badge',' post-card-inline__badge_default']})
categorias = [x.getText() for x in categorias]
_fechas = soup_page.find_all('time', {'class':['post-card-inline__date']})
_fechas = [x['datetime'] for x in _fechas]

Una vez se tienen todos los enlaces, se procede a tomar directamente el contenido de cada página

In [44]:
def extractor(url):
    page = scraper.get(url)
    sp = BeautifulSoup(page.text)
    p = sp.find_all('p')
    text = ''
    for r in p:
        text += r.getText()
    return text

In [45]:
import time
enlaces, contenido, fechas = [], [], []
for i in range(len(categorias)):
    if 'Noticias' in categorias[i]:
        time.sleep(3)
        try:
            enlaces.append(_links[i])
            contenido.append(extractor(_links[i]))
            fechas.append(_fechas[i])
        except Exception as e:
            if '404' in str(e):
                time.sleep(3)
                enlaces.append(_links[i])
                contenido.append(extractor(_links[i]))
                fechas.append(_fechas[i])
            else:
                print('{} \nEn enlace: {}'.format( str(e), _links[i] ) )
        

Finalmente, se guardan los datos obtenidos en un archivo csv

In [ ]:
import pandas as pd
data = pd.DataFrame({
    'Enlace': enlaces,
    'Texto': contenido,
    'Fecha': fechas
})
data.to_csv('datos.csv')

Con lo anterior, se obtendrán datos de mejor calidad de ahora en adelante más útiles para entrenar mejor al modelo.